In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
!sh run.sh vocab  ````                                        1 

In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
!python sanity_check.py 1e

/content/drive/My Drive/cs224n/assignment5
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
--------------------------------------------------------------------------------
Running Sanity Check for Question 1e: To Input Tensor Char
--------------------------------------------------------------------------------
Running test on a list of sentences
Sanity Check Passed for Question 1e: To Input Tensor Char!
--------------------------------------------------------------------------------


In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
from vocab import VocabEntry # class VocabEntery

sentences = [['Human', ':', 'What', 'do', 'we', 'want', '?'], ['Computer', ':', 'Natural', 'language', 'processing', '!'], ['Human', ':', 'When', 'do', 'we', 'want', 'it', '?'], ['Computer', ':', 'When', 'do', 'we', 'want', 'what', '?']]

ve = VocabEntry()
sents_var = ve.to_input_tensor_char(sents=sentences, device='cpu')

# (max_sentence_length, batch_size, max_word_length)
sents_var.shape # Expected shape (8, 4, 12)
#sents_var.is_contiguous() ?

/content/drive/My Drive/cs224n/assignment5


False

In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
from vocab import VocabEntry
from utils import pad_sents_char # function pad_sents_char
import torch

sentences = [['Human', ':', 'What', 'do', 'we', 'want', '?'], ['Computer', ':', 'Natural', 'language', 'processing', '!'], ['Human', ':', 'When', 'do', 'we', 'want', 'it', '?'], ['Computer', ':', 'When', 'do', 'we', 'want', 'what', '?']]
# len(sentences) = 4, batch_size = 4

ve = VocabEntry()
word_ids = ve.words2charindices(sents=sentences)
sents_padded = pad_sents_char(word_ids, ve.char_pad)

# (batch_size, max_sentence_length, max_word_length) = (4, 8, 12) before reshape
# (max_sentence_length, batch_size, max_word_length) = (8, 4, 12) after reshape
sents_var = torch.tensor(sents_padded, device='cpu').permute(1,0,2).contiguous()
print(sents_var.shape)
sent_var.is_contiguous()

/content/drive/My Drive/cs224n/assignment5
torch.Size([8, 4, 12])


True

In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
from highway import Highway
import torch
import torch.nn as nn

# We know that x_conv_out = MaxPool（ReLU (x_cov)) and
# that x_cov_out has a shape of (b, word_embed_size), where b is the batch size
# and word_embed_size is the size of final word embedding for word x.
# Having been through a ReLU function, the values within each x_conv_out 
# must be greater or equal to 0, and having been through the MaxPool function,
# these values are presumbly not too small.
# Using the above logic, we construct dummy test_x_conv_out with shape (3, 4) for sanity check,
# and we expect the output x_highway to have the same shape as input test_x_conv_out.
#  

print('====TEST HIGHWAY====')
print('---- Step-by-step Test----')

word_embed_size = 4
print('Use word_embed_size ', word_embed_size)

test_x_conv_out = torch.tensor([[0,0.1,0.9,100], [0.2,0.3,0.4,0.5], [2, 0.1, 5, 3]])
print('x_conv_out shape', test_x_conv_out.shape, ' expected [3, 4]')
print(test_x_conv_out)

w_proj = nn.Linear(word_embed_size, word_embed_size, bias=True) # W_project
x_proj = F.relu(w_proj(test_x_conv_out)) # (b, word_embed_size)
print('x_conv_out => x_proj, shape ', x_proj.shape, ' expected [3, 4]')

w_gate = nn.Linear(word_embed_size, word_embed_size, bias=True) # W_gate
x_gate = torch.sigmoid(w_gate(test_x_conv_out)) # (b, word_embed_size)
print('x_conv_out => x_gate, shape ', x_gate.shape, ' expected [3, 4]')

# element wise multiplication: * or mul()
x_highway = x_gate * x_proj + (1 - x_gate) * test_x_conv_out # (b, word_embed_size)
print('x_highway.shape ', x_highway.shape, ' expected [3, 4]')
print(x_highway)

print('----Class Test----')
print('Use word_embed_size 8')
# Use word_embed_size = 8, and generate test tensor accordingly.
hw = Highway(8)
dummy_x_conv_out = torch.rand(10, 8)
print('dummy_x_conv_out shape ', dummy_x_conv_out.shape, ' expected  [10, 8]')
x_highway = hw(dummy_x_conv_out)
print('x_highway shape ', x_highway.shape, ' expected [10, 8]')

/content/drive/My Drive/cs224n/assignment5
====TEST HIGHWAY====
---- Step-by-step Test----
Use word_embed_size  4
x_conv_out shape torch.Size([3, 4])  expected [3, 4]
tensor([[  0.0000,   0.1000,   0.9000, 100.0000],
        [  0.2000,   0.3000,   0.4000,   0.5000],
        [  2.0000,   0.1000,   5.0000,   3.0000]])
x_conv_out => x_proj, shape  torch.Size([3, 4])  expected [3, 4]
x_conv_out => x_gate, shape  torch.Size([3, 4])  expected [3, 4]
x_highway.shape  torch.Size([3, 4])  expected [3, 4]
tensor([[7.2133e-06, 1.0000e-01, 9.0000e-01, 4.1299e+01],
        [2.5218e-01, 1.6375e-01, 6.0326e-01, 2.6874e-01],
        [1.7667e+00, 4.3590e-01, 3.6010e+00, 9.4109e-01]],
       grad_fn=<AddBackward0>)
----Class Test----
Use word_embed_size 8
dummy_x_conv_out shape  torch.Size([10, 8])  expected  [10, 8]
x_highway shape  torch.Size([10, 8])  expected [10, 8]


In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
from cnn import CNN
import torch
import torch.nn as nn
import torch.nn.functional as F

"""
We know that the convolution layers takes input x_reshaped,
which has a shape of (b, f, m_word), where 
    b = batch size
    f = word_embed_size
    m_word = the length of longest word in the batch.
Given k = 5 and padding = 1, we create a dummy test tensor
with b = 4, char_embed_size = 8 and m_word = 10, in shape of (4,8,10),
and feed this test tensor through CNN with word_embed_size = 16

We expect the output from CNN to have a shape of
(b, word_embed_size) = (4, 16).

For intermediary steps, the following shapes are expected:
x_conv (b, word_embed_size, m_word + 2*padding - k +1) = (4, 16, 8)

"""
print('-------------------CNN TEST---------------------')
print('----Test Inputs----')
test_word_embed_size = 16
print('word_embed_size ', test_word_embed_size)
test_char_embed_size = 8
print('char_embed_size ', char_embed_size)
k = 5
print('k ', k)
b = 4
print('b ', b)
m_word = 10
print('m_word ', m_word)
padding = 1
print('padding ', padding)

print('--------STEP BY STEP TEST----------')
# x_reshaped in shape of (b, char_embed_size, m_word) 
test_x_reshaped = torch.rand(b, test_char_embed_size, m_word)
print('test_x_reshaped shape ', test_x_reshaped.shape, ' expected [4, 8, 10]') 

apply_conv = nn.Conv1d(in_channels=test_char_embed_size,
                    out_channels= test_word_embed_size, 
                    kernel_size= k, 
                    padding= padding
                    ) 
x_conv = apply_conv(test_x_reshaped)
print('x_conv shape ', x_conv.shape, ' expected [4, 16, 8]')

mp = nn.MaxPool1d(kernel_size = m_word + 2*padding -k +1)

x_conv_out = mp(F.relu(x_conv))
print('x_conv_out shape before squeeze on dim2', x_conv_out.shape, ' expected [4, 16, 1]')
x_conv_out = x_conv_out.squeeze(2)
print('x_conv_out shape after squeeze on dim2', x_conv_out.shape, ' expected [4, 16]')


print('--------CLASS TEST----------')
cnn = CNN(test_word_embed_size, test_char_embed_size, k, padding)
cnn_x_cov_out = cnn.forward(test_x_reshaped)

# Expected shape (b, word_embed_size, 1), after squeeze (b, word_embed_size)
print('\ncnn_x_cov_out shape ', cnn_x_cov_out.shape, ' expected [4, 16]') 
print(cnn_x_cov_out)

/content/drive/My Drive/cs224n/assignment5
-------------------CNN TEST---------------------
----Test Inputs----
word_embed_size  16
char_embed_size  50
k  5
b  4
m_word  10
padding  1
--------STEP BY STEP TEST----------
test_x_reshaped shape  torch.Size([4, 8, 10])  expected [4, 8, 10]
x_conv shape  torch.Size([4, 16, 8])  expected [4, 16, 8]
x_conv_out shape before squeeze on dim2 torch.Size([4, 16, 1])  expected [4, 16, 1]
x_conv_out shape after squeeze on dim2 torch.Size([4, 16])  expected [4, 16]
--------CLASS TEST----------

cnn_x_cov_out shape  torch.Size([4, 16])  expected [4, 16]
tensor([[0.3329, 0.0000, 0.5072, 0.0108, 1.0785, 0.3762, 0.6181, 0.4392, 0.0000,
         0.0398, 0.1480, 0.1031, 0.2878, 0.0000, 0.0000, 0.3743],
        [0.2789, 0.1419, 0.6266, 0.0603, 0.9503, 0.5482, 0.6596, 0.3022, 0.0000,
         0.1974, 0.1445, 0.1996, 0.2367, 0.0000, 0.0000, 0.3221],
        [0.2248, 0.0000, 0.5617, 0.0808, 0.8206, 0.4509, 0.7276, 0.3105, 0.0000,
         0.2695, 0.0380, 0.227

In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
from collections import Counter
from docopt import docopt
from itertools import chain
import json
import torch
from typing import List
from utils import read_corpus, pad_sents, pad_sents_char

sentences = [['Human', ':', 'What', 'do', 'we', 'want', '?'], ['Computer', ':', 'Natural', 'language', 'processing', '!'], ['Human', ':', 'When', 'do', 'we', 'want', 'it', '?'], ['Computer', ':', 'When', 'do', 'we', 'want', 'what', '?']]
sentence_length = 8
BATCH_SIZE = 4
word_length = 12
output = vocabEntry.to_input_tensor_char(sentences, 'cpu')
output_expected_size = [sentence_length, BATCH_SIZE, word_length]




In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
import torch
import torch.nn as nn

t = torch.rand(5,3,2)
print(t, '\n')
mp = nn.MaxPool1d(2)
mp(t)

/content/drive/My Drive/cs224n/assignment5
tensor([[[0.1912, 0.5272],
         [0.4881, 0.5688],
         [0.4425, 0.9390]],

        [[0.0742, 0.8956],
         [0.3391, 0.9755],
         [0.1352, 0.5569]],

        [[0.8652, 0.6584],
         [0.4846, 0.1898],
         [0.3727, 0.2502]],

        [[0.5535, 0.0803],
         [0.6066, 0.5563],
         [0.5278, 0.5386]],

        [[0.4833, 0.9264],
         [0.5839, 0.5302],
         [0.9279, 0.7697]]]) 



tensor([[[0.5272],
         [0.5688],
         [0.9390]],

        [[0.8956],
         [0.9755],
         [0.5569]],

        [[0.8652],
         [0.4846],
         [0.3727]],

        [[0.5535],
         [0.6066],
         [0.5386]],

        [[0.9264],
         [0.5839],
         [0.9279]]])

In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
from vocab import VocabEntry
from cnn import CNN
from highway import Highway
import torch
import torch.nn as nn

print('================MODEL_EMBEDDINGS TEST=================')
print('----Test Inputs----')
ve = VocabEntry()
num_char_embeddings = len(ve.char2id)
print('num_char_embeddings ', num_char_embeddings)
char_embed_size = 50
print('char_embed_size ', char_embed_size)
word_embed_size = 10
print('word_embed_size ', word_embed_size)
k = 5
print('k ', k)
padding = 1
print('padding ', padding)
test_dropout_rate = 0.3
print('dropout_rate ', test_dropout_rate)

"""
sents_var tensor size: (max_sentence_length, batch_size, max_word_length)
Start and end token added to each word, aka {word}, therefore
max_word_length = length of longest word + 2
"""
# test_sents list[list[str]]
test_sents = [['This', 'is', 'a', 'great', 'time', '.'],
              ['What', 'a', 'wonderful', 'day', '!'],
              ["Wait", 'a', 'minute', '!'],
              ['Goodbye', '.']
            ]
print('test sentences batch')
print(test_sents, '\n')
# batch_size = 4

print('----Step-by-step Test----')
sents_var = ve.to_input_tensor_char(test_sents, 'cpu')
print('[sentence_length, batch_size, max_word_length]')
print('sents_var shape ', sents_var.shape, ' expected [6 , 4, 11]')

CharEmbedding = nn.Embedding(num_embeddings = num_char_embeddings,
                             embedding_dim = char_embed_size,
                             )

# [max_sentence_length, b(sents), max_word_length, char_embed_size]
X_char_embedding = CharEmbedding(sents_var) 
X_char_embedding.shape # torch.Size([6, 4, 11, 50])
print("X_char_embedding shape ", X_char_embedding.shape, ' expected [6, 4, 11, 50]')

max_len_sents, b_sents, m_word, char_embed_size = X_char_embedding.shape
# reshape batch of sentences of sentences to batch of words
#   [max_len_sents*b_sents, m_word, char_embed_size] = [24, 11, 50])
X_reshaped = X_char_embedding.reshape(max_len_sents*b_sents, m_word, char_embed_size)
# permute to [max_len_sents*b_sents, char_embed_size, m_word]
X_reshaped = X_reshaped.permute(0, 2, 1).contiguous()
print('X_reshaped shape ', X_reshaped.shape, ' expected [24, 11, 50]')

# X_reshaped => cnn => X_conv_out
cnn = CNN(word_embed_size,char_embed_size, k, padding)
# X_conv_out: [b, word_embed_size]
X_conv_out = cnn(X_reshaped)
print('X_conv_out shape ', X_conv_out.shape, ' expected [24, 10]')

# X_conv_out => highway => X_highway
highway = Highway(word_embed_size)
# [b, word_embed_size]
X_highway = highway(X_conv_out)
print('X_highway shape ', X_highway.shape, ' expected [24, 10]')

# X_highway => dropout => X_word_embed
# X_word_embed: [b, word_embed_size]
dropout = nn.Dropout(test_dropout_rate)
X_word_embed = dropout(X_highway)
print('X_word_embed before reshape ', X_word_embed.shape, ' expected [24, 10]')
X_word_embed = X_word_embed.reshape(max_len_sents, b_sents, -1).contiguous()
print('X_word_embed after reshape ', X_word_embed.shape, ' expected [6, 4, 10]')

print('\n----Class Test----')
from model_embeddings import ModelEmbeddings
me = ModelEmbeddings(word_embed_size = word_embed_size, 
                     vocab = ve, 
                     char_embed_size = char_embed_size, 
                     k=k, 
                     padding=padding, 
                     dropout_rate=test_dropout_rate
                    ) 

test_input = ve.to_input_tensor_char(test_sents, 'cpu')
print('[sentence_length, batch_size, max_word_length]')
print('test_input size ', test_input.shape, ' expected [[6 , 4, 11]')
X_word_embed = me(test_input)
print('X_word_embed shape ', X_word_embed.shape, ' expected [6, 4, 10]')

/content/drive/My Drive/cs224n/assignment5
================MODEL_EMBEDDINGS TEST=================
----Test Inputs----
num_char_embeddings  96
char_embed_size  50
word_embed_size  10
k  5
padding  1
dropout_rate  0.3
test sentences batch
[['This', 'is', 'a', 'great', 'time', '.'], ['What', 'a', 'wonderful', 'day', '!'], ['Wait', 'a', 'minute', '!'], ['Goodbye', '.']] 

----Step-by-step Test----
[sentence_length, batch_size, max_word_length]
sents_var shape  torch.Size([6, 4, 11])  expected [6 , 4, 11]
X_char_embedding shape  torch.Size([6, 4, 11, 50])  expected [6, 4, 11, 50]
X_reshaped shape  torch.Size([24, 50, 11])  expected [24, 11, 50]
X_conv_out shape  torch.Size([24, 10])  expected [24, 10]
X_highway shape  torch.Size([24, 10])  expected [24, 10]
X_word_embed before reshape  torch.Size([24, 10])  expected [24, 10]
X_word_embed after reshape  torch.Size([6, 4, 10])  expected [6, 4, 10]

----Class Test----
[sentence_length, batch_size, max_word_length]
test_input size  torch.Size([

In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
!python sanity_check.py 1h

/content/drive/My Drive/cs224n/assignment5
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
--------------------------------------------------------------------------------
Running Sanity Check for Question 1h: Model Embedding
--------------------------------------------------------------------------------
Sanity Check Passed for Question 1h: Model Embedding!
--------------------------------------------------------------------------------


In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
!sh run.sh train_local_q1

/content/drive/My Drive/cs224n/assignment5
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
uniformly initialize parameters [-0.100000, +0.100000]
use device: cpu
begin Maximum Likelihood training
epoch 2, iter 10, avg. loss 105.52, avg. ppl 111.11 cum. examples 20, speed 176.67 words/sec, time elapsed 2.54 sec
epoch 4, iter 20, avg. loss 100.53, avg. ppl 88.95 cum. examples 40, speed 221.90 words/sec, time elapsed 4.55 sec
epoch 6, iter 30, avg. loss 97.23, avg. ppl 76.77 cum. examples 60, speed 215.77 words/sec, time elapsed 6.63 sec
epoch 8, iter 40, avg. loss 93.60, avg. ppl 65.27 cum. examples 80, speed 241.67 words/sec, time elapsed 8.48 sec
epoch 10, iter 50, avg. loss 90.48, avg. ppl 56.79 cum. examples 100, speed 232.62 words/sec, time elapsed 10.41 sec
epoch 12, iter 60, avg. loss 87.41, avg. ppl 49.51 cum. examples 120, speed 211.60 words/sec, time elapsed 12.53 sec
epoch 14, iter 70, avg. loss 85.06, avg. ppl 44.

In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
!sh run.sh test_local_q1

/content/drive/My Drive/cs224n/assignment5
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
load test source sentences from [./en_es_data/test_tiny.es]
load test target sentences from [./en_es_data/test_tiny.en]
load model from model.bin
Decoding:   0% 0/4 [00:00<?, ?it/s]/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
Decoding: 100% 4/4 [00:00<00:00, 17.72it/s]
Corpus BLEU: 99.66941696422141


In [0]:
t = torch.tensor([1,2,3,4,5,6, 7, 8, 9, 10, 11, 12])
t = t.reshape(2,2,3)
t

tensor([[[ 1,  2,  3],
         [ 4,  5,  6]],

        [[ 7,  8,  9],
         [10, 11, 12]]])

In [0]:
t = t.reshape(4, 3)
t

tensor([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12]])

In [0]:
t = t.reshape(2,2,3)
t

tensor([[[ 1,  2,  3],
         [ 4,  5,  6]],

        [[ 7,  8,  9],
         [10, 11, 12]]])

In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
import torch
import torch.nn as nn
from sanity_check import DummyVocab

"""
init INPUTS
hidden_size, char_embedding_size=50, target_vocab=None
"""

print('----Test Inputs----')
BATCH_SIZE = 5
char_embedding_size = 3
hidden_size = 4
DROPOUT_RATE = 0.0
sequence_length = 4
inpt = torch.zeros(sequence_length, BATCH_SIZE, dtype=torch.long)

target_vocab=DummyVocab()
dec_hidden=None

print('----Step-by-step Test----')
charDecoder = nn.LSTM(char_embedding_size, hidden_size)
char_output_projection = nn.Linear(hidden_size, len(target_vocab.char2id))
decoderCharEmb = nn.Embedding(len(target_vocab.char2id), 
                              char_embedding_size,
                              padding_idx=target_vocab.char_pad
                              )

# input: [length, b] ==> decoderCharEmb => X: [length, b, char_embed_size]
X = decoderCharEmb(inpt)
print ("X shape", X.shape, ' expected [4, 5, 3]')

# X: [length, b, char_embed_size], dec_hidden = (h_n, c_n): ([1, b, h], [1, b, h])
#   ==> charDecoder ==> 
# h_t: [length, b char_embed_size], dec_hidden = (h_n, c_n): ([1, b, h], [1, b, h])
h_t, dec_hiddens = charDecoder(X, dec_hidden)
print('h_t shape ', h_t.shape, ' expected [4, 5, 3]')

# dec_hidden ==> char_output_projection ==> scores: [length, b, self.vocab_size]
scores = char_output_projection(h_t)
print ('scores shape ', scores.shape, ' expected [4, 5,', len(target_vocab.char2id), ']')


/content/drive/My Drive/cs224n/assignment5
----Test Inputs----
----Step-by-step Test----
X shape torch.Size([4, 5, 3])  expected [4, 5, 3]
h_t shape  torch.Size([4, 5, 4])  expected [4, 5, 3]
scores shape  torch.Size([4, 5, 32])  expected [4, 5, 32 ]


In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
!python sanity_check.py 2a

/content/drive/My Drive/cs224n/assignment5
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
--------------------------------------------------------------------------------
Running Sanity Check for Question 2a: CharDecoder.forward()
--------------------------------------------------------------------------------
Sanity Check Passed for Question 2a: CharDecoder.forward()!
--------------------------------------------------------------------------------


In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
import torch
import torch.nn as nn
import torch.nn.functional as F
from char_decoder import CharDecoder
from sanity_check import DummyVocab
from vocab import VocabEntry

"""
init INPUTS
hidden_size, char_embedding_size=50, target_vocab=None
"""

print('='*10, 'CharDecoder.train_forward() Test', '='*10)
print('----Test Inputs----')
BATCH_SIZE = 5
char_embedding_size = 3
hidden_size = 4
DROPOUT_RATE = 0.0
sequence_length = 4
target_vocab=DummyVocab()
dec_hidden=None
char_sequence = torch.zeros(sequence_length, BATCH_SIZE, dtype=torch.long)
print('char_sequence shape ', char_sequence.shape, ' expected [4, 5]')

# char_sequence: [length, b] => delete end_token => input_sequence: [length, b]
X_input = char_sequence[:-1]
print('X_input shape ', X_input.shape, 'expected [3, 5]')

# char_sequence: [length, b] => delete start_token => input_sequence: [length, b]
X_target = char_sequence[1:]
print('X_target shape ', X_target.shape, 'expected [3, 5]')

cd = CharDecoder(hidden_size, char_embedding_size, target_vocab)
# X_input: [length, b], dec_hidden = (h_n, c_n): ([1, b, h], [1, b, h])
#    ==> softmax   ==>
# s_t: [length, b, self.vocab_size], dec_hidden = (h_n, c_n): ([1, b, h], [1, b, h])
s_t, dec_hidden = cd.forward(X_input, dec_hidden)
print('s_t shape', s_t.shape, ' expected [3, 5,', len(target_vocab.char2id), ']')

# For lookup char_pad index value, shall be 0
ve = VocabEntry()
idx_char_pad = ve.char_pad

# Initialiate CrossEntropyLoss Instances, combines logsoftmax and nllloss
compute_loss = nn.CrossEntropyLoss(ignore_index = idx_char_pad,
                                    reduction ='sum'
                                  )

# Reshape s_t for compute_loss, length*b => b_char
# length = length of a word, b = batch size, length*b = # of characters in the batch
# s_t: [length, b, self.vocab_size] ==> s_t: [length*b, self.vocab_size] = [N, C]
s_t = s_t.reshape(s_t.shape[0]*s_t.shape[1], -1)

# Reshape X_target for compute_loss
# X_target: [length, b] ==> X_target: [length*b] = [N]
X_target = X_target.reshape(-1)

# s_t: [length*b, self.vocab_size] = [N, C, d1...dk], X_target: [length*b] = [N]
#   ==> compute_loss ==> loss_char_dec: 
loss_char_dec = compute_loss(s_t, X_target) 

/content/drive/My Drive/cs224n/assignment5


SyntaxError: ignored

In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
import torch.nn.functional as F
t = torch.rand(6, 4, 10)
print(t.shape)
F.softmax(t, dim=2).shape

/content/drive/My Drive/cs224n/assignment5
torch.Size([6, 4, 10])


torch.Size([6, 4, 10])

In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
!python sanity_check.py 2b

/content/drive/My Drive/cs224n/assignment5
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
--------------------------------------------------------------------------------
Running Sanity Check for Question 2b: CharDecoder.train_forward()
--------------------------------------------------------------------------------
Sanity Check Passed for Question 2b: CharDecoder.train_forward()!
--------------------------------------------------------------------------------


In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
import torch
x = torch.rand(2,3,4)哪句8
t = (x, x)
t[0].shape[1]

In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
import torch

from vocab import VocabEntry
from sanity_check import DummyVocab
from char_decoder import CharDecoder
import torch.nn.functional as F
import numpy as np
import time

print('='*10, 'CharDecoder.decode_greedy() Test', '='*10)
print('----Test Inputs----')
BATCH_SIZE = 5
char_embedding_size = 50
hidden_size = 4
DROPOUT_RATE = 0.0
target_vocab=DummyVocab()


# Initialize decode_greedy()
dummy_t1 = torch.randint(2,20, (1, BATCH_SIZE, hidden_size))
dummy_t2 = torch.zeros(1, BATCH_SIZE, hidden_size)

dummy_t3 = torch.rand(1, BATCH_SIZE, hidden_size)
dummy_t3_sum = dummy_t3.sum()
dummy_t3 = dummy_t3 / dummy_t3_sum

dummy_t4 = torch.rand(1, BATCH_SIZE, hidden_size)
dummy_t4_sum = dummy_t4.sum()
dummy_t4 = dummy_t4 / dummy_t4_sum

initialStates = (dummy_t3, dummy_t3)
device = 'cpu'
max_length=21 # Preset maximum output_word length

# initialStates: ([1, batch_size, hidden_size], [1, batch_size, hidden_size])
#    ==> read ==> batch_size: int
batch_size = initialStates[0].shape[1]
print('batch_size ', batch_size, ', expected', BATCH_SIZE)

# iniitalStates ==> dec_hidden = (h0, c0): [1, batch_size, hidden_size], [1, batch_size, hidden_size]
dec_hidden = initialStates

###################################
start_time = time.time()
###################################

# Initialize output_word as an empty, output_word (Tensor): [length <= max_length = 0, batch_size]
output_word = torch.empty(0, batch_size, dtype=torch.long , device='cpu')
print('output_word', output_word)

# Initialize current_char (Tensor): [1, b]
ve = VocabEntry()
current_char = torch.tensor([ve.start_of_word]*batch_size).reshape(1, -1).contiguous()
print('current_char shape', current_char.shape, ', expected [1, {}]'.format(batch_size))

cd = CharDecoder(hidden_size, char_embedding_size, target_vocab)

for i in range(0, max_length-1):
# current_char: [1, b], dec_hidden = (h_n, c_n): ([1, b, h], [1, b, h])
#    ==> self.forward   ==>
# s_t: [1, b, self.vocab_size], dec_hidden = (h_n, c_n): ([1, b, h], [1, b, h])
  s_t, dec_hidden = cd.forward(current_char, dec_hidden)

  # s_t: [length=1, b, self.vocab_size] ==> softmax ==> p_t: [length=1, b, self.vocab.size]
  p_t = F.softmax(s_t, dim=2)
  #print('p_t shape', p_t.shape, ', expected [1, {}, {}]'.format(batch_size, 'vocab_size'))

  # p_t: [length=1, b, self.vocab_size] ==> argmax ==> current_char: [length=1, b]
  current_char = torch.argmax(p_t, dim=2)
  #print('current_char shape', current_char.shape, ', expected [1, {}]'.format(batch_size))

  # current_char (Tensor): [1, b]  ==>  output_word (Tensor) [length <= max_length, b]
  output_word = torch.cat((output_word, current_char), dim=0)
  #print("output_word shape ", output_word.shape)

# output_word (Tensor) [max_length, b] ==> output_word (List(List)): [b, max_lengnth]
#output_word = np.array(output_word).T.tolist()
output_word = output_word.permute(1,0).tolist()
print(output_word)

# Trucate each word in batch starting from the first end_of_word token <END>='}'
# output_word (List(List[int]): [b, max_lengnth] ==> output_word (List(List(int))): [b, length <= max_length]
output_word = [cids[0:cids.index(ve.end_of_word)] if ve.end_of_word in cids else cids for cids in output_word]

# Convert character indices to characters
# output_word (List(List[int])):  [b, length <= max_length] ==> (List(List[str])): [b, length <= max_length, 1]
decodedWords = [[ve.id2char[cid] for cid in word] for word in output_word]

# output_word (List(List[str])): [b, length <= max_length, 1]
#     ==> decodedWords (List[str]): [b, length <= max_length]
decodedWords = [''.join(char) for char in decodedWords]

print(decodedWords)
print("--- %s seconds ---" % (time.time() - start_time))

/content/drive/My Drive/cs224n/assignment5
========== CharDecoder.decode_greedy() Test ==========
----Test Inputs----
batch_size  5 , expected 5
output_word tensor([], size=(0, 5), dtype=torch.int64)
current_char shape torch.Size([1, 5]) , expected [1, 5]
[[29, 29, 29, 30, 29, 30, 30, 21, 20, 20, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23], [29, 29, 29, 30, 29, 30, 30, 21, 20, 20, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23], [29, 29, 29, 30, 29, 30, 30, 21, 20, 20, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23], [29, 29, 29, 30, 29, 30, 30, 21, 20, 20, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23], [29, 29, 29, 30, 29, 30, 30, 21, 20, 20, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23]]
['ZZZaZaaRQQTTTTTTTTTT', 'ZZZaZaaRQQTTTTTTTTTT', 'ZZZaZaaRQQTTTTTTTTTT', 'ZZZaZaaRQQTTTTTTTTTT', 'ZZZaZaaRQQTTTTTTTTTT']
--- 0.008855581283569336 seconds ---


In [0]:
import numpy as np
t = [[1, 4, 5, 6, 2, 8, 2, 2], [5, 4, 2, 6, 7, 8, 2, 2]]
ve.end_of_word
t = [[ve.id2char[wid] for wid in word] for word in t]
[''.join(word) for word in t]

['{ABC}E}}', 'BA}CDE}}']

In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
!python sanity_check.py 2c

/content/drive/My Drive/cs224n/assignment5
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
--------------------------------------------------------------------------------
Running Sanity Check for Question 2c: CharDecoder.decode_greedy()
--------------------------------------------------------------------------------
Sanity Check Passed for Question 2c: CharDecoder.decode_greedy()!
--------------------------------------------------------------------------------


In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
!sh run.sh train_local_q2

/content/drive/My Drive/cs224n/assignment5
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
uniformly initialize parameters [-0.100000, +0.100000]
use device: cpu
begin Maximum Likelihood training
epoch 2, iter 10, avg. loss 478.64, avg. ppl 1904824833.40 cum. examples 20, speed 153.70 words/sec, time elapsed 2.91 sec
epoch 4, iter 20, avg. loss 371.05, avg. ppl 15628238.21 cum. examples 40, speed 196.43 words/sec, time elapsed 5.20 sec
epoch 6, iter 30, avg. loss 351.52, avg. ppl 6535443.23 cum. examples 60, speed 190.95 words/sec, time elapsed 7.54 sec
epoch 8, iter 40, avg. loss 344.75, avg. ppl 4831755.59 cum. examples 80, speed 210.71 words/sec, time elapsed 9.67 sec
epoch 10, iter 50, avg. loss 339.44, avg. ppl 3811048.72 cum. examples 100, speed 212.40 words/sec, time elapsed 11.78 sec
epoch 12, iter 60, avg. loss 334.24, avg. ppl 3021898.13 cum. examples 120, speed 190.22 words/sec, time elapsed 14.13 sec
epoch 14, i

In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
!sh run.sh test_local_q2

/content/drive/My Drive/cs224n/assignment5
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
load test source sentences from [./en_es_data/test_tiny.es]
load test target sentences from [./en_es_data/test_tiny.en]
load model from model.bin
Decoding:   0% 0/4 [00:00<?, ?it/s]/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
Decoding: 100% 4/4 [00:00<00:00,  6.58it/s]
Corpus BLEU: 99.66941696422141


In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
!sh run.sh train

Streaming output truncated to the last 5000 lines.
epoch 22, iter 147490, avg. loss 95.81, avg. ppl 99.16 cum. examples 47680, speed 6276.33 words/sec, time elapsed 16049.21 sec
epoch 22, iter 147500, avg. loss 88.69, avg. ppl 79.40 cum. examples 48000, speed 6367.87 words/sec, time elapsed 16050.22 sec
epoch 22, iter 147510, avg. loss 99.08, avg. ppl 93.99 cum. examples 48320, speed 6488.28 words/sec, time elapsed 16051.30 sec
epoch 22, iter 147520, avg. loss 94.04, avg. ppl 96.28 cum. examples 48640, speed 6374.06 words/sec, time elapsed 16052.33 sec
epoch 22, iter 147530, avg. loss 83.32, avg. ppl 64.68 cum. examples 48960, speed 5565.33 words/sec, time elapsed 16053.48 sec
epoch 22, iter 147540, avg. loss 80.62, avg. ppl 64.48 cum. examples 49280, speed 5425.60 words/sec, time elapsed 16054.62 sec
epoch 22, iter 147550, avg. loss 88.41, avg. ppl 79.52 cum. examples 49600, speed 5809.61 words/sec, time elapsed 16055.74 sec
epoch 22, iter 147560, avg. loss 91.15, avg. ppl 86.23 cum. 

In [0]:
%cd /content/drive/My Drive/cs224n/assignment5
!sh run.sh test

/content/drive/My Drive/cs224n/assignment5
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
load test source sentences from [./en_es_data/test.es]
load test target sentences from [./en_es_data/test.en]
load model from model.bin
Decoding:   0% 0/8064 [00:00<?, ?it/s]/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
Decoding: 100% 8064/8064 [08:17<00:00, 16.21it/s]
Corpus BLEU: 36.380266115259616
